In [1]:
import pandas as pd
import numpy as np

In [2]:
from fbprophet import Prophet

In [3]:
from fbprophet.plot import add_changepoints_to_plot
from matplotlib import pyplot as plt


In [4]:
data=pd.read_csv('../input/hourly-data/hourly_data.csv')

In [5]:
data.head()

In [6]:
data=data[['date_time','AC 1']]

In [7]:
 data.rename(columns={'date_time':'ds','AC 1':'y'} ,inplace=True)

In [8]:
data.head()

In [9]:
train=data[data['ds'] <='2019-09-15']
test=data[data['ds'] >'2019-09-15']

In [10]:
m=Prophet(changepoint_prior_scale=2.5, weekly_seasonality = True)

In [11]:
m.fit(train)

In [12]:
future=m.make_future_dataframe(periods=384 , freq='H')
forecast= m.predict(future)

In [13]:
forecast.tail()

In [14]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [15]:
m.plot(forecast)


In [16]:
m.plot_components(forecast)


In [17]:
forecast.head()

In [18]:
forecast.shape

In [19]:
data.shape

In [20]:
pred= data.copy()

In [21]:
pred['yhat']=forecast['yhat']

In [22]:
pred.plot()

In [23]:
# basic model of fb prophet fails to capture the peak points
# sampling the data to weekly might give better results
# we can add temperature variables as well
# we can add holidays as well
# we can use weekends flag as well

In [24]:
data=pd.read_csv('../input/hourly-data/hourly_data.csv')

In [25]:
data.columns

In [26]:
data=data[['date_time','AC 1','humidity','holiday_flag','tempC']]

In [27]:
 data.rename(columns={'date_time':'ds','AC 1':'y'} ,inplace=True)

In [28]:
data.head()

In [29]:
train=data[data['ds'] <'2019-09-15']
test=data[data['ds'] >'2019-09-15']

In [30]:
prophet = Prophet(growth='linear',
                  yearly_seasonality=False,
                  weekly_seasonality=True,
                  daily_seasonality=False,
                  holidays=None,
                  seasonality_mode='multiplicative',
                  seasonality_prior_scale=7,
                  holidays_prior_scale=5,
                  changepoint_prior_scale=.05,
                  mcmc_samples=0
                 ).add_seasonality(name='weekly',
                                    period=168,
                                    fourier_order=2,
                                    prior_scale=10,
                                    mode='multiplicative')

In [31]:
prophet.add_country_holidays(country_name='IN')
prophet.add_regressor('humidity')
prophet.add_regressor('holiday_flag')
prophet.add_regressor('tempC')
prophet.fit(train)


In [32]:
future = prophet.make_future_dataframe(periods=384, freq='H')


In [33]:
future['humidity'] = data['humidity']
future['holiday_flag'] = data['holiday_flag']
future['tempC'] = data['tempC']
forecast = prophet.predict(future)
fig = prophet.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), prophet, forecast)
plt.show()
fig2 = prophet.plot_components(forecast)
plt.show()

In [34]:
data['yhat']= forecast['yhat']

In [35]:
data.set_index('ds',inplace=True)

In [36]:
data[['y','yhat']].tail(360).plot()

In [37]:
15*24